### Assignment-03

- Formulate three original hypotheses based on the raw US Accidents dataset, and perform appropriate statistical hypothesis tests to validate each one.

In [1]:
!pip install gdown

In [2]:
import gdown

url = "https://drive.google.com/uc?id=1XXgJNTEl4p4APOAYsDXEFEk9XX-8lg07"
gdown.download(url, "accidents.csv", quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1XXgJNTEl4p4APOAYsDXEFEk9XX-8lg07
From (redirected): https://drive.google.com/uc?id=1XXgJNTEl4p4APOAYsDXEFEk9XX-8lg07&confirm=t&uuid=121ec17b-05d9-4dab-b907-a2916480af48
To: /content/accidents.csv
100%|██████████| 3.06G/3.06G [00:54<00:00, 56.5MB/s]


'accidents.csv'

In [1]:
import pandas as pd

chunks = pd.read_csv("/content/accidents.csv", chunksize=200000)
df = pd.concat(chunks)

Hypothesis 1: Effect of Precipitation on Accident Severity

Precipitation (rain, snow, sleet) reduces road friction and visibility, which may increase crash severity. This helps understand weather-driven risk escalation.

H₀ (Null Hypothesis):
Mean accident severity is the same for accidents with precipitation and without precipitation.

H₁ (Alternative Hypothesis):
Mean accident severity differs between accidents with precipitation and without precipitation.

Test Used : Welch’s t-test (Independent two-sample t-test)

In [2]:
from scipy.stats import ttest_ind

# Create precipitation indicator
precip = df[df['Precipitation(in)'] > 0]['Severity']
no_precip = df[df['Precipitation(in)'] == 0]['Severity']

# Remove missing values
precip = precip.dropna()
no_precip = no_precip.dropna()

# Two-sample t-test
stat, p_val = ttest_ind(precip, no_precip, equal_var=False)

alpha = 0.05
print(f"T-statistic: {stat:.3f}, p-value: {p_val:.4g}")

if p_val < alpha:
    print("Reject H0: Mean severity differs with precipitation.")
else:
    print("Fail to reject H0: No significant difference in severity.")

#Reject H₀: Precipitation significantly affects accident severity
#Fail to reject H₀: Severity is not significantly influenced by precipitation


T-statistic: 142.504, p-value: 0
Reject H0: Mean severity differs with precipitation.


Hypothesis 2: :Junction vs Non-Junction Accident Severity

Is accident severity higher at junctions than on regular road segments?
Here Junctions means:Conflicting traffic movements, Turns and signal violations

H₀: Mean severity at junctions equals mean severity at non-junctions.
H₁: Mean severity at junctions is higher than at non-junctions.

Test Used: One-sided independent t-test

In [3]:
from scipy.stats import ttest_ind

junction = df[df['Junction'] == True]['Severity']
non_junction = df[df['Junction'] == False]['Severity']

junction = junction.dropna()
non_junction = non_junction.dropna()

# One-sided test: junction severity > non-junction severity
stat, p_val = ttest_ind(junction, non_junction, equal_var=False)

alpha = 0.05
print(f"T-statistic: {stat:.3f}, p-value: {p_val/2:.4g}")

if (p_val/2 < alpha) and (stat > 0):
    print("Reject H0: Junctions have higher accident severity.")
else:
    print("Fail to reject H0: No evidence that junctions are more severe.")
#Reject H₀: Junctions are more dangerous
#Fail to reject H₀: No significant severity difference

T-statistic: 126.007, p-value: 0
Reject H0: Junctions have higher accident severity.


Hypothesis 3: Traffic Signal Presence and Accident Severity Association

Are accidents equally likely to occur with and without traffic signals?
Traffic signals are meant to reduce conflicts.
This tests whether signals are statistically associated with accident occurrence.

H₀: Accident occurrence is independent of traffic signal presence.
H₁: Accident occurrence is dependent on traffic signal presence.

Test Used: Chi-Square Test of Independence


In [4]:
from scipy.stats import chi2_contingency

# Create contingency table
contingency = pd.crosstab(df['Traffic_Signal'], df['Severity'] >= 3)

chi2, p_val, dof, expected = chi2_contingency(contingency)

alpha = 0.05
print(f"Chi-square statistic: {chi2:.3f}, p-value: {p_val:.4g}")

if p_val < alpha:
    print("Reject H0: Accident severity is associated with traffic signal presence.")
else:
    print("Fail to reject H0: No association found.")
#Reject H₀: Traffic signals are linked to accident severity patterns
#Fail to reject H₀: Severity is independent of signal presence

Chi-square statistic: 84836.523, p-value: 0
Reject H0: Accident severity is associated with traffic signal presence.
